In [1]:
import requests
import json
import pandas as pd

In [11]:
# NAVITIME APIキーを設定 
API_KEY = '----'

In [5]:
# 保育園データの読み込み
df = pd.read_csv("0224_hoikuen.csv")
# ninka_data = df[df["区分"].str.contains("認可保育園")]

# 保育園の緯度経度を作成(１列にまとめている)
df["latlng"] = df.apply(lambda x:'{},{}'.format(x["latitude"], x["longitude"]), axis=1)
# df["latlng"]

In [6]:
# 保育園データ数の検索
print(len(df["latlng"]))

197


In [9]:
# 保育園の最寄り駅を検索
url = "https://navitime-transport.p.rapidapi.com/transport_node/around"

headers = {
	"X-RapidAPI-Key": API_KEY,
	"X-RapidAPI-Host": "navitime-transport.p.rapidapi.com"
}

for i in range(len(df)):
    querystring = {
        "coord":df["latlng"][i],
        "limit":"1",
        "term":"60",
        "datum":"wgs84",
        "coord_unit":"degree",
        "walk_speed":"5"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)

    # pythonの辞書型に変換
    res = json.loads(response.text)
    
    df.loc[i,"nearlest_station"] = res["items"][0]["name"]
    df.loc[i,"walk_time"] = res["items"][0]["time"]
    
    # print(i,res["items"][0]["name"],res["items"][0]["time"])
    

0 石川台 10
1 緑が丘（東京都） 5
2 上野毛 8
3 二子玉川 12
4 用賀 15


In [10]:
df.head()

,区分,名称,住所,0歳定員,1歳定員,2歳定員,3歳定員,4歳定員,5歳定員,定員,...,対象年齢,開所時間,閉所時間,延長保育時間（開始）,延長保育時間（終了）,latitude,longitude,latlng,nearlest_station,walk_time
0,認可保育園,南奥沢☆,東京都世田谷区奥沢1−2−13,NaN,14.0,14.0,17.0,17.0,17.0,79,...,NaN,7:15:00,18:15:00,18:15:00,20:15:00,35.599453,139.680111,"35.5994527,139.6801106",石川台,10.0
1,認可保育園,奥沢☆,東京都世田谷区奥沢2−3−11,10.0,10.0,11.0,17.0,17.0,18.0,83,...,NaN,7:15:00,18:15:00,18:15:00,20:15:00,35.605270,139.675704,"35.6052698,139.6757044",緑が丘（東京都）,5.0
2,認可保育園,中町※1,東京都世田谷区中町3−27−2,11.0,16.0,18.0,19.0,20.0,21.0,105,...,NaN,7:15:00,18:15:00,18:15:00,20:15:00,35.615462,139.642539,"35.6154624,139.6425391",上野毛,8.0
3,認可保育園,玉川,東京都世田谷区玉川4−16−6,NaN,15.0,18.0,20.0,22.0,22.0,97,...,NaN,7:15:00,18:15:00,18:15:00,20:15:00,35.617734,139.624131,"35.6177344,139.6241309",二子玉川,12.0
4,認可保育園,上用賀☆,東京都世田谷区上用賀4−2−10,9.0,14.0,15.0,21.0,22.0,22.0,103,...,NaN,7:15:00,18:15:00,18:15:00,20:15:00,35.634891,139.630393,"35.6348913,139.6303933",用賀,15.0
